## Supreme Court Opinion Getter
This tiny amount of code grabs a random sample of Supreme Court Opinions for the entire history of the court. This project has a ton of parts, so I haven't included the code I wrote to get the links to these cases.

In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

In [359]:
supcourt = pd.read_pickle("supcourt_yearlist.pickle")
test_df = supcourt.sample(10000, random_state=2).reset_index()

In [360]:
def supcourtopinions():
    links = test_df.caseurl
    info = {}
    
    for link in links:
        response = requests.get(link)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        pagesoup = soup.find_all(class_="caselawcontent searchable-content") 
        for items in pagesoup:
            text = items.get_text()
            #print(text)
             #random thing I figured out which separates opinions
            if re.search("onwrit of certiorari to", text):
                souplist = text.split("on writ of certiorari to")
                souplist.pop(0)
                for item in souplist:      
                    match = re.search("Justice.........\s?", item)
                    if match:
                        justice = match.group().strip("Justice")
                        justice = justice.split(' ')[1].strip(',').lower()
                        info[link] = [justice]
                        info[link].append(item)
            else:
                match = re.search("Justice.........\s?", text)
                if match:
                    justice = match.group().strip("Justice")
                    justice = justice.split(' ')[1].strip(',').lower()
                    info[link] = [justice]
                    info[link].append(text)
            
    return info   

def supcourtdescr(lnks):
    links = [lnks]
    
    for link in links:
        response = requests.get(link)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        pagesoup = soup.find_all(class_="caselawcontent searchable-content") 
     
        for item in pagesoup:
            text = item.get_text()
            souplist = text.split("on writ of certiorari to") #random thing I figured out which separates opinions
            description = souplist[0]
    return description   

data = supcourtopinions()

In [361]:
test = pd.DataFrame(data).transpose().reset_index()
test.columns = ["caseurl", "justice", "assocopinion"]
test_full = pd.merge(test,test_df, on="caseurl", how = "inner")

In [362]:
test_full["description"] = test_full.caseurl.apply(supcourtdescr)

In [ ]:
test_full2 = test_full
def cleanerup(text):
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("\\", " ")
    return(text)

def cleanupcase(text):
    text = re.sub("[^a-zA-Z. ]", "", str(text).lower())
    return text
test_full2["casetitle"] = test_full2.casetitle.apply(cleanerup)
test_full2["assocopinion"] = test_full2.assocopinion.apply(cleanerup)
test_full2["description"] = test_full2.description.apply(cleanerup)
test_full2["casetitle"] = test_full2.casetitle.apply(cleanupcase)
test_full2.sort(["casetitle"], ascending = True)
#test_full2.to_csv("test2.csv")

## Cleaning up Justices - first pass
The way I had to extract the justice names from the opinions was not perfect, but it was the only way that worked. Aforementioned, I figured out that generally the opinions were separated by "on write critorai", and then found that generally the first mention of the word Justice was followed by the Justices' name that wrote the opinion (I cross referenced this with a cursory wikipedia check about how many opinions each justice wrote, and I believe it to be accurate). Anyway, I did collect some random words with this process, so I created a stopword dict to pass through the df. I will pass through a list of the justice names spelled correctly to compile all of them properly.

In [364]:
df3 = pd.read_csv("junk.csv")
stopwords = list(df3.iloc[:,0])

#cleaning out the nonjustice names from the dataframe
def remove_stopwords(text):
    if text in stopwords:
        return ' '
    else:
        return text.strip(".").strip("'s").strip("\n").strip(":").strip(",")
test_full2["justice"] = test_full2.justice.apply(remove_stopwords)
test_full2 = test_full2[test_full2["justice"] != " "]

## Removing Footnotes
This is the dirtiest data I've ever worked with. There doesn't appear to be a source for these opinions that separates them clearly. What ends up happening is that some unknown percentage ends up with the footnotes in it (and we also don't know whih those are), so we need to clean those out.

In [ ]:
def stripfootnotes(text):
    if re.search("FOOTNOTES", text):
        textlist = text.split("FOOTNOTES")
        return textlist[0]
    else:
        return text
test_full2["assocopinion"] = test_full2.assocopinion.apply(stripfootnotes)    

In [366]:
test_full2 = test_full2[['caseurl', 'justice', 'assocopinion', 'casetitle', 'years',
       'description']]

## Cleaning up Justice Names - Final Pass
I came up with a creative solution for this. I had found a wiki page that listed every single justice in the history of justices. I took those names and cross referenced it with my first pass names. I saved the first pass names as a csv and manually put their true alias in the column next to that. Then I reopened that file, merged it with the full dataframe and deleted the first pass column. This would have obviously not been possible if there weren't only about 100 justices, but it worked for this project.


In [369]:
test_full2.head(5).to_csv("supcourtsamp.csv")

In [383]:
test_full2.justice.value_counts().to_csv("justicesforcleanup.csv")
justicecleanup = pd.read_csv("justicesforcleanup.csv")
justicecleanup.columns = ["justice", "cleaned"]
mergd = pd.merge(test_full2, justicecleanup, how="inner", on = "justice")

In [388]:
mergd = mergd[['caseurl', 'cleaned', 'assocopinion', 'casetitle', 'years',
       'description']]
mergd.to_pickle("supcourtsample2.pkl")

In [164]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords ##Note you'll need to download NLTK and corpuses
from spacy.en import English ##Note you'll need to install Spacy and download its dependencies
parser = English()

In [230]:
supcourt_decoy = pd.read_pickle("supcourtsample2.pkl")

In [231]:

def cleanerup(text):
    text = text.replace(",", " ")
    text = text.replace(".", " ")
    dump, keep = re.split("Argued:", text)
    #text = text.replace("United States Supreme Court", " ")
    keep = re.sub("[^a-zA-Z. ""]", "", str(keep).lower())
    return keep

supcourt_decoy["assocopinion"] = supcourt_decoy.assocopinion.apply(cleanerup)

In [232]:

supcourt_decoy.to_pickle("supcourtsample_cleaned.pkl")

In [ ]:
supcourt["assocopinion"] = supcourt.assocopinion.apply(cleanerup)
supcourt

In [290]:
#some random words I added to the stopword dictionary that are legalese 

def stopwordz(text):
    STOPLIST = set(stopwords.words('english') + ["n't", ' ', "'s", "a", "aan", "aba", "aand", "of", "m", "u", "f", 
                                                 "j", "juan", "ca", "u.s.", "u.", 'aa', 'aaa', 'aab', 'aabd', 'aac', 
                                                 'aag', 'aai', 'aaii', 'aaiii', 'aalthough', 'zosenjo', 'zschernig'])
    words = text.split(' ')
    texts = [word for word in words if word not in STOPLIST]
    #if texts not ' ':
    return ' '.join(texts)


In [ ]:
supcourt_decoy["assocopinion2"] = supcourt_decoy.assocopinion.apply(stopwordz)

In [263]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
def lemmatize(text):
    tokens = parser(text)
    wordsa = [word.lemma_ for word in tokens if word != "-PRON-"]
    return ' '.join(wordsa)
supcourt_decoy["assocopinion3"] = supcourt_decoy.assocopinion2.apply(lemmatize)

In [265]:
supcourt_decoy.ix[16, 'assocopinion3']
supcourt_decoy.to_pickle("supcourt_lemmatized.pkl")

## Condensing Opinions
I needed a 'body of work' document for each justice, so I grouped each opinion by the justice who wrote it

In [284]:
#supcourt2 = supcourt2.dropna()
supcourt2 = supcourt_decoy[["cleaned", "assocopinion3"]]
supcourt2 = supcourt2.groupby(["cleaned"]).sum()



In [ ]:
supcourt2['assocopinion3'] = supcourt2.assocopinion3.apply(stopwordz)
supcourt2

In [292]:
supcourt2.to_pickle("collapsedjustices_lemmatized.pickle")

In [1]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [298]:
vectorizer = CountVectorizer(min_df = 5)
dtm = vectorizer.fit_transform(supcourt2.assocopinion3) 

lsa = TruncatedSVD(5, algorithm = 'randomized')
dtm_lsa = lsa.fit_transform(dtm)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
lsa.explained_variance_ratio_

array([ 0.81160062,  0.07032553,  0.02812444,  0.01206823,  0.00725964])

In [ ]:
# Get words that correspond to each column 
vectorizer.get_feature_names()

In [ ]:
similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T) 

In [ ]:
pd.DataFrame(dtm.toarray(), index=supcourt2.assocopinion3, columns=vectorizer.get_feature_names()).head(1)

# Putting it all together with a model
since the sources of the data were very different, I had to get both dataframes into a similar format and put them together. Below, I am opening two pickles for the justices and the nominee. I put the nominee in with the analysis because this isn't truly a 'prediction' since we are clustering.

In [323]:
supcourt2 = pd.read_pickle("collapsedjustices_lemmatized.pickle")
garland = pd.read_pickle("garland_lemmatized.pickle")
garland = garland[["type", 'text']]
garland = garland[garland['type'] == "constitutional-law"]
garland = garland.groupby(["type"]).sum()
def join(text):
    text = ' '.join(text)
    text = text.replace(',', '')
    text = text.replace('.', ' ')
    return text
garland['text'] = garland.text.apply(join)

In [330]:
garland.reset_index(inplace = True)

In [332]:
garland.columns = ["type", "assocopinion3"]

In [335]:
garland['cleaned'] = 'GARLAND'
garland = garland[['assocopinion3', 'cleaned']]

In [340]:
supcourt2.reset_index(inplace = True)
test = pd.concat([supcourt2, garland])

,assocopinion3,cleaned
0,decide may motion petitioner leave pr...,ALITO
1,decide june july appellees comm...,BARNARD
2,december decide january ...,BLACK
3,decide october application vacate st...,BLACKMUN
4,decide february august united s...,BRADLEY
5,november decide december m...,BRANDEIS
6,decide may messrs sylvan c balde...,BRENNAN
7,decide february march pa...,BREWER
8,decide october petition writ certiora...,BREYER
9,january decide february petition ...,BROWN


In [342]:
test.to_pickle('totalproject.pkl')
supcourt2 = test

In [343]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction

In [10]:
#a stemmer helps us pick up speech patterns and condense the matrix down to a more accurate format
#it gives you the root of a word (EX, give from gives)

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [11]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [344]:
#supcourt2 = pd.read_pickle("collapsedjustices_lemmatized.pickle")
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, 
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))


tfidf_matrix = tfidf_vectorizer.fit_transform(supcourt2.assocopinion3) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

(75, 88331)


In [345]:
terms = tfidf_vectorizer.get_feature_names()

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
dist

array([[ -2.22044605e-16,   9.83458460e-01,   6.95980197e-01, ...,
          7.59474246e-01,   8.59261949e-01,   9.26899677e-01],
       [  9.83458460e-01,  -6.66133815e-16,   9.54766330e-01, ...,
          9.78331870e-01,   9.84003110e-01,   6.94635021e-01],
       [  6.95980197e-01,   9.54766330e-01,  -4.66293670e-15, ...,
          7.22172893e-01,   8.03646046e-01,   8.17265670e-01],
       ..., 
       [  7.59474246e-01,   9.78331870e-01,   7.22172893e-01, ...,
         -4.44089210e-16,   8.96053530e-01,   9.32282203e-01],
       [  8.59261949e-01,   9.84003110e-01,   8.03646046e-01, ...,
          8.96053530e-01,   6.66133815e-16,   9.29298098e-01],
       [  9.26899677e-01,   6.94635021e-01,   8.17265670e-01, ...,
          9.32282203e-01,   9.29298098e-01,  -2.22044605e-15]])

## Modeling - K-means
I ultimately settled on a K-means clustering because it was the algorithm with the least variablility between each running of the grouping.

In [346]:
from sklearn.cluster import KMeans
num_clusters = 4
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [347]:
#this is just the easiest way I know to do to check freq of items in a list
clustcount = pd.Series(clusters)
clustcount.value_counts()
#clustcount
supcourt2['clusters'] = pd.Series(clusters, index=supcourt2.index)
#supcourt2
#you HAVE to have the index number for the index in the series for it to work. Otherwise it NANs

In [372]:
supcourt2.cleaned.unique #making sure this workded

<bound method IndexOpsMixin.unique of 0            ALITO
1          BARNARD
2            BLACK
3         BLACKMUN
4          BRADLEY
5         BRANDEIS
6          BRENNAN
7           BREWER
8           BREYER
9            BROWN
10          BURGER
11          BURTON
12          BUTLER
13           BYRNE
14        CAMPBELL
15         CARDOZO
16           CHASE
17           CLARK
18          CLARKE
19        CLIFFORD
20          CURTIS
21             DAY
22         DOUGLAS
23          FORTAS
24     FRANKFURTER
25          FULLER
26        GINSBURG
27        GOLDBERG
28            GRAY
29          HARLAN
          ...     
45          NELSON
46        O'CONNOR
47        PATERSON
48         PECKHAM
49          PITNEY
50          POWELL
51            REED
52       REHNQUIST
53         ROBERTS
54        RUTLEDGE
55         SANFORD
56          SCALIA
57          SHIRAS
58       SOTOMAYOR
59          SOUTER
60         STEVENS
61         STEWART
62           STONE
63           STORY
64      SUTH

## Pulling in extra info
I put together the justicesummary pickle in the notebook supcourt_justices_wiki.ipynb in this repo. As a review - I pulled that info from wikipedia, it is a table with summary info about each justice that I am now appending to my main dataframe in the below cells. I'm doing this because it helps with interpretability and "learn more" for other people - this is the main info that contributes to the hover over summaries in the viz for this project

In [373]:
df = pd.read_pickle('justicesummary.pickle')
df.columns = ['dummy', 'name', 'State', 'lifetime', 'service', 'ChiefJus', 'Retired',
       'Appointedby', 'termination', 'cleaned', 'serviceyrs', 'start', 'end']

#some extra info collection for a later time - this is whether or not the justice was a chief
def Chiefjus(text):
    if text is not '':
        return 1
    else:
        return 0

df['ChiefJus'] = pd.DataFrame.replace(df['ChiefJus'], to_replace= '—', value = '')
df.ChiefJus = df.ChiefJus.apply(Chiefjus) 


In [377]:
mergd = pd.merge(supcourt2, df, how='outer', on = 'cleaned')
mergd.cleaned.to_csv('w.csv')

In [378]:
mergd['cleaned'] = pd.DataFrame.replace(mergd['cleaned'], to_replace= '', value = np.nan)
mergd = mergd[mergd['cleaned'] != np.nan]

mergd = mergd[['cleaned', 'assocopinion3', 'clusters',
       'name', 'State', 'lifetime', 'ChiefJus', 'Retired',
       'Appointedby', 'termination', 'start', 'end']]
mergd

,cleaned,assocopinion3,clusters,name,State,lifetime,ChiefJus,Retired,Appointedby,termination,start,end
0,ALITO,decide may motion petitioner leave pr...,1,"Alito, SamuelSamuel Alito",NJ,195004011950–,0,—,"Bush, G. W.",Currently serving,2006,now
1,BARNARD,decide june july appellees comm...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BLACK,december decide january ...,3,"Black, HugoHugo Black",AL,188602271886–1971,1,197109171971–1971 (0 years),"Roosevelt, F.",Retirement[n 6],1937,1971
3,BLACKMUN,decide october application vacate st...,1,"Blackmun, HarryHarry Blackmun",MN,190811121908–1999,0,199408031994–1999 (4–5 years),Nixon,Retirement[n 6],1970,1994
4,BRADLEY,decide february august united s...,3,"Bradley, Joseph PhiloJoseph P. Bradley",NJ,181303141813–1892,0,—,Grant,Death,1870,1892
5,BRANDEIS,november decide december m...,3,"Brandeis, Louis DembitzLouis Brandeis",MA,185611131856–1941,0,193902131939–1941 (1–2 years),Wilson,Retirement[n 6],1916,1939
6,BRENNAN,decide may messrs sylvan c balde...,1,"Brennan, William J.William J. Brennan, Jr.*",NJ,190604251906–1997,0,199007201990–1997 (6–7 years),Eisenhower,Retirement[n 6],1956,1990
7,BREWER,decide february march pa...,2,"Brewer, David JosiahDavid Josiah Brewer",KS,183706201837–1910,0,—,"Harrison, B.",Death,1890,1910
8,BREYER,decide october petition writ certiora...,1,"Breyer, StephenStephen Breyer",MA,193808151938–,0,—,Clinton,Currently serving,1994,now
9,BROWN,january decide february petition ...,2,"Brown, Henry BillingsHenry Billings Brown",MI,183603021836–1913,0,—,"Harrison, B.",Retirement,1891,1906


In [388]:
from collections import Counter
def verbose(text):
    text = re.sub("[^a-zA-Z ""]", "", str(text).lower())
    strng = len(text)
    return strng

def uniques(text):
    text = re.sub("[^a-zA-Z ""]", "", str(text).lower())
    text2 = text.split(' ')
    counted_docs = Counter(text2)
    return len(counted_docs.keys())

mergd['uniquewords'] = mergd.assocopinion3.apply(uniques)
mergd['verbose'] = mergd.assocopinion3.apply(verbose)

### Creating a verboseness measure for size of bubbles in viz

In [389]:
mergd['verboseratio'] = mergd['uniquewords'] / mergd['verbose'] * 100

In [390]:
parties = pd.read_csv('presidents_forpiv.csv',encoding = 'iso-8859-1')
parties.columns = ['index', 'Appointedby', 'party']

In [391]:
mergd2 = pd.merge(mergd, parties, on= 'Appointedby', how = 'outer')
mergd2.to_pickle('cluster_justices_withparties.pickle')

In [360]:
supcourt2.clusters.value_counts() #I tried a number of iterations for how many clusters I'd do, optimized at 4

1    27
2    21
3    15
0    12
Name: clusters, dtype: int64